In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
from glom import glom
from datetime import timedelta

### Heart Rate Signal 

In [2]:
df = pd.read_json("/Users/olumide/Documents/Dissertation/Sound Stimuli version 1/MyFitbitData version 2/OlumideOdetunde/Physical Activity/heart_rate-2022-06-15.json") #load data

# extract each value
df_bpm = df['value'].apply(lambda row:glom(row,"bpm"))
df_confidence = df["value"].apply(lambda row:glom(row, "confidence"))
df_datetime = df["dateTime"]

#combine extracted for required timeframe
df_hr = pd.concat([df_datetime, df_bpm, df_confidence], axis=1, ignore_index=True)
df_hr.columns = ["datetime","heartrate","confidence"] #rename column
df_hr.set_index("datetime",inplace=True)
df_hr = df_hr.loc["2022-06-15 18:45:01": "2022-06-15 20:00:01"]
df_hr.reset_index(inplace=True)

#print and save to local directory 
df_hr.to_csv('/Users/olumide/Documents/Dissertation/Sound Stimuli version 1/Extracted data/hr.csv')
print("signal successfully saved")
df_hr.head()

signal successfully saved


,datetime,heartrate,confidence
0,2022-06-15 18:45:05,70,3
1,2022-06-15 18:45:10,73,3
2,2022-06-15 18:45:25,75,3
3,2022-06-15 18:45:30,74,3
4,2022-06-15 18:45:35,73,3


### Heartrate Variability

In [3]:
df_hrv =pd.read_csv("/Users/olumide/Documents/Dissertation/Sound Stimuli version 1/MyFitbitData version 2/OlumideOdetunde/Sleep/Heart Rate Variability Details - 2022-06-10.csv")
df_hrv["timestamp"] =pd.to_datetime(df_hrv.timestamp)
df_hrv.rename(columns={"timestamp":"datetime"}, inplace=True)

#print and save to local directory
df_hrv.to_csv('/Users/olumide/Documents/Dissertation/Sound Stimuli version 1/Extracted data/hrv.csv')
print("signal successfully saved")
df_hrv.head()

signal successfully saved


,datetime,rmssd,coverage,low_frequency,high_frequency
0,2022-06-10 01:50:00,48.441,0.918,76.398,321.501
1,2022-06-10 01:55:00,54.408,0.969,323.342,407.796
2,2022-06-10 02:00:00,70.821,0.997,175.632,681.564
3,2022-06-10 02:05:00,72.079,0.958,179.117,706.678
4,2022-06-10 02:10:00,66.215,0.983,139.691,756.989


### ECG Reading

In [4]:
df_ecg = pd.read_csv("/Users/olumide/Documents/Dissertation/Sound Stimuli version 1/MyFitbitData version 2/OlumideOdetunde/Heart/Afib ECG Readings.csv")
df_ecg["reading_time"] =pd.to_datetime(df_ecg.reading_time)

df_ecg.to_csv('/Users/olumide/Documents/Dissertation/Sound Stimuli version 1/Extracted data/ecg.csv')
print("signal successfully saved")
df_ecg.head()

signal successfully saved


,reading_id,reading_time,wire_id,result_classification,heart_rate,heart_rate_alert,firmware_version,device_app_version,hardware_version,waveform_samples
0,24acf5e0-e4f8-11ec-8080-808080808080,2022-06-05 17:51:35+00:00,497267d1532c,NSR,63,NONE,128.6.12,2.9.0,Sense,[-32768 -32768 -32768 -32768 -32768 -3276...
1,cc22f1a0-e325-11ec-8080-808080808080,2022-06-03 10:13:21+00:00,497267d1532c,NSR,60,NONE,128.6.12,2.9.0,Sense,[32767 32767 32767 32767 32767 32767 327...


### Skin Temperature Signal

In [5]:

df_skt = pd.read_csv("/Users/olumide/Documents/Dissertation/Sound Stimuli version 1/MyFitbitData version 2/OlumideOdetunde/Sleep/Wrist Temperature - 2022-06-15.csv")

#extract values for required timeframe
df_skt.columns = ["datetime","temperature"]
df_skt["datetime"] = pd.to_datetime(df_skt['datetime'], format="%Y-%m-%dT%H:%M:%S.%f", errors = 'coerce') #changes iso dateformat
df_skt.set_index("datetime", inplace=True)
df_skt = df_skt.loc["2022-06-15 18:41":"2022-06-15 20:00"]
df_skt.reset_index(inplace=True)

#print and save to local directory
df_skt.to_csv('/Users/olumide/Documents/Dissertation/Sound Stimuli version 1/Extracted data/skintemp.csv')
print("signal successfully saved")
df_skt.head()


signal successfully saved


,datetime,temperature
0,2022-06-15 18:41:00,-6.460064
1,2022-06-15 18:42:00,-6.415064
2,2022-06-15 18:43:00,-6.415064
3,2022-06-15 18:44:00,-1.945064
4,2022-06-15 18:45:00,-1.330064


### Blood Oxygen variation

In [6]:
df_bo = pd.read_csv("/Users/olumide/Documents/Dissertation/Sound Stimuli version 1/MyFitbitData version 2/OlumideOdetunde/Other/estimated_oxygen_variation-2022-06-15.csv")
df_bo["timestamp"] =pd.to_datetime(df_bo.timestamp)
df_bo.rename(columns={"timestamp":"datetime"}, inplace=True)
df_bo.set_index("datetime", inplace=True)
df_bo = df_bo.loc["2022-06-15 18:41":"2022-06-15 20:00"]
df_bo.reset_index(inplace=True)

#print and save to local directory
df_bo.to_csv('/Users/olumide/Documents/Dissertation/Sound Stimuli version 1/Extracted data/bo.csv')
print("signal successfully saved")
df_bo.tail()

signal successfully saved


,datetime,Infrared to Red Signal Ratio
46,2022-06-15 19:55:31,-27
47,2022-06-15 19:56:32,-12
48,2022-06-15 19:57:32,0
49,2022-06-15 19:58:32,0
50,2022-06-15 19:59:32,-2


### Respiratory signals

In [48]:
#load and reanme timestamp column
df_rsp = pd.read_csv("/Users/olumide/Documents/Dissertation/Sound Stimuli version 1/MyFitbitData version 2/OlumideOdetunde/Sleep/Respiratory Rate Summary - 2022-06-01.csv")
df_rsp["timestamp"] =pd.to_datetime(df_rsp.timestamp)
df_rsp.rename(columns={"timestamp":"datetime"}, inplace=True)

#save to local directory
df_rsp.to_csv('/Users/olumide/Documents/Dissertation/Sound Stimuli version 1/Extracted data/rsp.csv')
print("signal successfully saved")
df_rsp.head()


signal successfully saved


,datetime,full_sleep_breathing_rate,full_sleep_standard_deviation,full_sleep_signal_to_noise,deep_sleep_breathing_rate,deep_sleep_standard_deviation,deep_sleep_signal_to_noise,light_sleep_breathing_rate,light_sleep_standard_deviation,light_sleep_signal_to_noise,rem_sleep_breathing_rate,rem_sleep_standard_deviation,rem_sleep_signal_to_noise
0,2022-06-06 04:09:30,19.0,1.1,6.882,19.0,1.1,6.882,18.4,1.4,8.618,-1.0,0.0,2.466
1,2022-06-07 04:18:30,19.2,1.0,5.776,19.2,1.0,5.776,19.0,1.5,5.594,-1.0,0.0,1.794
2,2022-06-08 08:06:30,20.0,0.8,6.799,20.0,0.8,6.799,18.4,2.1,9.753,-1.0,0.0,1.717
3,2022-06-10 07:27:00,18.4,1.2,8.001,18.4,1.2,8.001,17.8,0.9,10.869,17.2,0.6,2.813


### Merging physiological signals

In [38]:
print(df_bo.shape)
print(df_skt.shape)

(51, 2)
(79, 2)


In [39]:
df_bo.head()

,datetime,Infrared to Red Signal Ratio
0,2022-06-15 19:09:30,-9
1,2022-06-15 19:10:30,0
2,2022-06-15 19:11:30,-6
3,2022-06-15 19:12:30,-12
4,2022-06-15 19:13:30,0


In [40]:
df_skt.head()

,datetime,temperature
0,2022-06-15 18:41:00,-6.460064
1,2022-06-15 18:42:00,-6.415064
2,2022-06-15 18:43:00,-6.415064
3,2022-06-15 18:44:00,-1.945064
4,2022-06-15 18:45:00,-1.330064


In [49]:

merged_dataframe = pd.merge_asof(df_skt, df_bo, on="datetime",
                                 direction="nearest", tolerance=pd.Timedelta(seconds=30),allow_exact_matches=True)
merged_dataframe.tail()
                                 

,datetime,temperature,Infrared to Red Signal Ratio
74,2022-06-15 19:55:00,1.464936,-8.0
75,2022-06-15 19:56:00,1.489936,-27.0
76,2022-06-15 19:57:00,1.499936,-12.0
77,2022-06-15 19:58:00,1.494936,0.0
78,2022-06-15 19:59:00,1.489936,0.0


## Dataset Alignment and Creation

### Load Data Labels

In [9]:
df_datalabel = pd.read_excel("/Users/olumide/Documents/Dissertation/Sound Stimuli version 1/Excel Database.xlsx")
df_datalabel.head()

,SoundID,Starttime,Emotion,Emotionrating,Endtime,UserID,Soundgroup,Soundgroupselection
0,0319_2,15/06/2022 18:51:47,Fear,5,15/06/2022 18:51:58,1,Practice,Practice Sound
1,0617_2,15/06/2022 18:52:00,Fear,4,15/06/2022 18:52:14,1,Practice,Practice Sound
2,0124_2,15/06/2022 18:52:16,Happiness,4,15/06/2022 18:52:32,1,Practice,Practice Sound
3,0533_2,15/06/2022 18:52:35,Sadness,4,15/06/2022 18:52:49,1,Practice,Practice Sound
4,0194_2,15/06/2022 18:52:50,Fear,5,15/06/2022 18:53:07,1,Practice,Practice Sound


### Aligning signals and labels


In [34]:
#create a datetime that is six seconds from the starttime
df_datalabel['Starttime'] = pd.to_datetime(df_datalabel.Starttime, format='%d/%m/%Y %H:%M:%S')
df_datalabel['Endtime'] = pd.to_datetime(df_datalabel.Endtime, format='%d/%m/%Y %H:%M:%S')
df_datalabel["datetime"] = df_datalabel["Starttime"] + timedelta(seconds = 6)
df_datalabel["difference"] = df_datalabel["Endtime"] - df_datalabel["datetime"] #differnce in sconds bettwen end of listening and submission
print(df_datalabel["difference"].median())
df_datalabel.head()

0 days 00:00:06


,SoundID,Starttime,Emotion,Emotionrating,Endtime,UserID,Soundgroup,Soundgroupselection,datetime,difference
0,0319_2,2022-06-15 18:51:47,Fear,5,2022-06-15 18:51:58,1,Practice,Practice Sound,2022-06-15 18:51:53,0 days 00:00:05
1,0617_2,2022-06-15 18:52:00,Fear,4,2022-06-15 18:52:14,1,Practice,Practice Sound,2022-06-15 18:52:06,0 days 00:00:08
2,0124_2,2022-06-15 18:52:16,Happiness,4,2022-06-15 18:52:32,1,Practice,Practice Sound,2022-06-15 18:52:22,0 days 00:00:10
3,0533_2,2022-06-15 18:52:35,Sadness,4,2022-06-15 18:52:49,1,Practice,Practice Sound,2022-06-15 18:52:41,0 days 00:00:08
4,0194_2,2022-06-15 18:52:50,Fear,5,2022-06-15 18:53:07,1,Practice,Practice Sound,2022-06-15 18:52:56,0 days 00:00:11


In [35]:
#adding an extra column to hr dataframe to track the merging
df_hr["hr_timestamp"] = df_hr["datetime"]
df_hr.head()
df_hr.head()

,datetime,heartrate,confidence,hr_timestamp
0,2022-06-15 18:45:05,70,3,2022-06-15 18:45:05
1,2022-06-15 18:45:10,73,3,2022-06-15 18:45:10
2,2022-06-15 18:45:25,75,3,2022-06-15 18:45:25
3,2022-06-15 18:45:30,74,3,2022-06-15 18:45:30
4,2022-06-15 18:45:35,73,3,2022-06-15 18:45:35


In [36]:
merged_dataframe = pd.merge_asof(df_datalabel, df_hr, on="datetime",
                                 direction="forward", tolerance=pd.Timedelta(seconds=6),allow_exact_matches=True)

In [37]:
merged_dataframe.to_csv('/Users/olumide/Documents/Dissertation/Sound Stimuli version 1/Extracted data/ merged_df5.csv')
print("merged data succesfully saved")
merged_dataframe.head()

merged data succesfully saved


,SoundID,Starttime,Emotion,Emotionrating,Endtime,UserID,Soundgroup,Soundgroupselection,datetime,difference,heartrate,confidence,hr_timestamp
0,0319_2,2022-06-15 18:51:47,Fear,5,2022-06-15 18:51:58,1,Practice,Practice Sound,2022-06-15 18:51:53,0 days 00:00:05,NaN,NaN,NaT
1,0617_2,2022-06-15 18:52:00,Fear,4,2022-06-15 18:52:14,1,Practice,Practice Sound,2022-06-15 18:52:06,0 days 00:00:08,75.0,2.0,2022-06-15 18:52:10
2,0124_2,2022-06-15 18:52:16,Happiness,4,2022-06-15 18:52:32,1,Practice,Practice Sound,2022-06-15 18:52:22,0 days 00:00:10,75.0,3.0,2022-06-15 18:52:25
3,0533_2,2022-06-15 18:52:35,Sadness,4,2022-06-15 18:52:49,1,Practice,Practice Sound,2022-06-15 18:52:41,0 days 00:00:08,74.0,3.0,2022-06-15 18:52:45
4,0194_2,2022-06-15 18:52:50,Fear,5,2022-06-15 18:53:07,1,Practice,Practice Sound,2022-06-15 18:52:56,0 days 00:00:11,NaN,NaN,NaT
